In [ ]:
!pip install openai==1.12.0
!pip install langchain==0.1.6
!pip install tiktoken==0.6.0
# !pip install faiss-gpu==1.7.2
!pip install faiss-cpu==1.8.0

In [1]:
import json, openai, mlflow
from mlflow.metrics.metric_definitions import _token_count_eval_fn
from langchain.llms import AzureOpenAI
from langchain.llms import OpenAI
from langchain.chat_models import AzureChatOpenAI
import pandas as pd
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.chains import RetrievalQA

In [2]:
experiment_name = "SAS"mlflow.create_experiment(experiment_name, artifact_location="s3://my-byte/Joel/ByteMLFlow")

mlflow.set_experiment(experiment_name)

# mlflow.langchain.autolog(log_input_examples=True,
#                          log_model_signatures=True,
#                          log_models=False,
#                          log_datasets=True,
#                          log_inputs_outputs=True,
#                          disable=False,
#                          exclusive=False,
#                          disable_for_unsupported_versions=True,
#                          silent=False,
#                          extra_tags=None)
# mlflow.langchain.LOG_MODEL_PARAM_DOCS

<Experiment: artifact_location='file:///newvolume/mybyte/Joel/Byte/mlruns/235239485690937724', creation_time=1711365066701, experiment_id='235239485690937724', last_update_time=1711365066701, lifecycle_stage='active', name='SAS', tags={}>

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="embedding_deployment",
    model="text-embedding-ada-002",
    api_version="2023-09-15-preview",  # give right one
    openai_api_type="azure",
    azure_endpoint="https://pharma-ml-poc-1.openai.azure.com/",
)

/newvolume/mybyte/Joel/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
azure_engine = "GPT4Turbo1106Preview"
azure_openai_api_type = "azure"
azure_openai_api_base = "https://pnlp-janseen.openai.azure.com/"
azure_openai_api_version = "2023-12-01-preview"


# Use environment variable for API key

azurellm = AzureChatOpenAI(
    model = 'gpt-4-1106-preview',
    # model = 'gpt-4-vision-preview',
    temperature=0,
    max_tokens=4096,
    openai_api_type=azure_openai_api_type,
    openai_api_base=azure_openai_api_base,
    openai_api_version=azure_openai_api_version,
    openai_api_key=azure_openai_api_key,
    deployment_name=azure_engine
    # azure_engine=azure_engine
)



/newvolume/mybyte/Joel/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/newvolume/mybyte/Joel/venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://pnlp-janseen.openai.azure.com/ to https://pnlp-janseen.openai.azure.com/openai.
  warnings.warn(
/newvolume/mybyte/Joel/venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:

In [5]:
import pandas as pd
df = pd.read_excel('content/other 334 (3).xlsx')
# df.head()

In [6]:
#try using one prompt - RAG

import time
from time import sleep

summary = []
content = []
name = []
summary_time = []
author = ''
publication_year = ''


text_splitter = CharacterTextSplitter(
        separator = "\n",
        chunk_size = 1000,
        chunk_overlap = 500,
        length_function = len,
    )


for index, _ in df.iterrows():
#   if index in [0]:
    input_data = []
    content.append(df['full text'][index])
    name.append(df['name'][index])
    author = df['author'][index]
    print("author -- ",author)
    publication_year = df['publication_year'][index]
    # author_and_year = df['Author(Publication_Year)'][index]
    # print("author_and_year -- ",author_and_year)
    start_time = time.time()
    cite_info = f"{author} ({publication_year})"

    input_text = df['full text'][index]
    texts = text_splitter.split_text(input_text)
    print(len(texts))

    docsearch = FAISS.from_texts(texts, embeddings)


    end_time = time.time()
    embeddings_time_taken = end_time - start_time
    print(f"embeddings time: {embeddings_time_taken} seconds")


    

    # set up FAISS as a generic retriever
    retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":50})

    rqa = RetrievalQA.from_chain_type(llm=azurellm,
                                      retriever=retriever,
                                      return_source_documents=True,


    )
    start_time = time.time()

    summary_t3_dev_final_prompt = f"""Consider yourself as a medical writer.
    Your job is to summarize the given input text like a medical writer summary.
    The output summary must have 4 sections: Background,efficacy,safety and author conclusion sections.
    Must use the following information for author_name(publication_year): {cite_info}


    Background section:
    Must include the author name and the publication year in the following format: 'author_name(publication_year)'
    Must begin with "author_name(publication_year)" .Strictly not to make any changes in the author_name(publication_year).
    Include essential details like study aim,study design,randomization method,control,blinding,population,diagnosis of interest,diagnostic criteria,age,sex,treatment and exposure,number of participants per treatment,disease condition,dose,dosage form,dose frequency,mode of administration,treatment duration,efficacy endpoints,safety endpoints.
    Only include the mentioned essential details if they are available.
    Include the following details only if present: The total number of subjects that were included in the analysis(with split-ups),subset of subjects that were specifically analyzed for safety-related outcomes or parameters(with split-ups), subset of subjects that were analyzed for efficacy-related outcomes(with split-ups) maintaining their verbatim representation.
    Strictly Important: If any detail is not present/explicitly present, the prompt should ignore it, and there should be no generation of model content, such as 'The study does not specify the country where it was conducted'
    Do not explicitly mention the names of the essential details in the output summary like 'The diagnosis of interest was', 'The study design was'.
    Highly Important: If the aim or study design is not explicitly presented, generate and include them in the background section.
    Must retain the intervention details like treatment regimen with dose in the output summary.
    Strictly important to include the demographics of study participants(including mean/median age,percentage of participants split),treatment regimen,inclusion and exclusion criteria as originally presented in the input text.
    Avoid including information about how the study is conducted or the study procedure.
    Include information about medical procedure.
    Highly Important: Include the detailed information(only if available) about how the randomization process was conducted,the method used to assign patients to different treatment arms, maintaining their verbatim representation.
    Must include the data enclosed with brackets.

    Efficacy section:
    Extract the efficacy results including the results of the efficacy endpoints/effectiveness evaluation and present it in an elaborated manner.
    Highly Important: Include percentages, absolute values (if available), p-values(if available), confidence intervals(if available), odds ratio(if available), hazard ratio(if available), relative risk(if available). Must include the data enclosed within brackets.

    Safety section:
    Extract the safety results including the results of the safety endpoints/safety evaluation and present it in an elaborated manner.
    Highly Important: Include percentages, absolute values (if available), p-values(if available), confidence intervals(if available), odds ratio(if available), hazard ratio(if available), relative risk(if available). Must include the data enclosed within brackets.

    Author conclusion:
    Read the entire input text and find out the author conclusion.
    When presenting the author conclusion, include the phrase 'The authors concluded that' at the beginning.
    Highly Important: If the author conclusion is not present explicitly, generate abstractively based on understanding.

    ***Strictly adhere to the following guidelines:***
    Highly important to make sure not to include the section names in the output summary.
    Highly important to make sure not to include headers, groups and number listing in the output summary.
    Highly important to make sure not to give introduction sentence,conclusion sentence while generating summary.
    Safety results should follow the presentation of efficacy results, and the author conclusion should be presented at the end.
    Highly Important: Reproduce sentences in the paragraph mostly verbatim to maintain language flow.
    Ensure there is no repetition of information between different parts of the output summary.
    Strictly avoid including reference information like references to tables, paragraphs, figures, etc.
    Do not add newline characters in any other part of the output summary.
    Incorporate very concise form of study limitations from the input text into the output summary, only if available. Don't generate on your own.
    Organize the information in chronological order.
    Strictly avoid providing any false information.
    Highly important to differentiate the efficacy,safety and author conclusion sections by adding a newline character('\n\n').
"""

    summary_qa = rqa(summary_t3_dev_final_prompt)

    end_time = time.time()
    summary_time_taken = end_time - start_time
    print(f"summary time: {summary_time_taken} seconds")

    # s_qa = summary_qa
    summary_output = summary_qa['result']
    print("summary_output",summary_output)



    summary.append(summary_output)
    summary_time.append(summary_time_taken)

xl_pd = pd.DataFrame()
pd.set_option('display.max_columns', None)
xl_pd = pd.DataFrame()

xl_pd["article"] = name
xl_pd["content"] = content
xl_pd["summary"] = summary
xl_pd["Time taken"] = summary_time

xl_pd['prompt_token'] = _token_count_eval_fn([summary_t3_dev_final_prompt]).scores[0]
xl_pd['content_token'] = xl_pd['content'].apply(lambda x: _token_count_eval_fn([x]).scores[0])
xl_pd['summary_token'] = xl_pd['summary'].apply(lambda x: _token_count_eval_fn([x]).scores[0])
xl_pd['Total_token'] = xl_pd['prompt_token'] + xl_pd['content_token'] + xl_pd['summary_token']


author --  Aussems and Van Cutsem
10
embeddings time: 0.500269889831543 seconds


/newvolume/mybyte/Joel/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


summary time: 29.576965808868408 seconds
summary_output Aussems and Van Cutsem (1974) conducted an open trial involving 157 outpatients with 176 lesions of apparently fungal origin associated with inflammatory symptoms. The study aimed to evaluate the efficacy and safety of Daktacort cream, which contains miconazole nitrate 2% and hydrocortisone 1%. Patients applied the cream twice daily to the affected areas for a median duration of four weeks. The clinical results were scored as cure (++), improvement (+), and insufficient clinical results (0). The population included patients with lesions ranging from four days to ten years, with a history of various previous treatments. The diagnosis was confirmed by direct microscopical examination and culture of skin scrapings, except in cases where the fungal origin was clinically evident. Concomitant oral treatment with miconazole nitrate was given to four patients.

The efficacy of the treatment was notable, with 83% (130 out of 157) of patien

Created a chunk of size 1320, which is longer than the specified 1000
Created a chunk of size 1237, which is longer than the specified 1000


summary time: 23.45547318458557 seconds
summary_output Rieth (1973) conducted a study to evaluate the therapeutic efficacy and local tolerance of miconazole plus, a combination of 2% miconazole nitrate cream and 1% hydrocortisone, in the treatment of various dermatomycoses. The study was an open-label trial involving 36 out-patients, with an age range from 8 to 87 (mean 54 years), consisting of 18 males and 18 females. Diagnoses were confirmed microscopically using the KOH-method and by cultural identification on Kimmig-agar medium. The treatment regimen included the application of miconazole plus topically twice a day until complete healing, with 20 cases also using 2% miconazole powder once or twice a day. No other antifungal medications were used during the study. The efficacy endpoints were the clinical condition and extension of the lesion, with a complete cure defined as a 100% reduction of the lesion. Out of the 33 patients who completed the study, 32 achieved complete cure.

Th

Created a chunk of size 1203, which is longer than the specified 1000


summary time: 26.75906229019165 seconds
summary_output Staal (1985) conducted a study to evaluate the efficacy and tolerability of Daktacort fatty ointment in patients with inflammatory dermatomycosis. The study was designed to include patients with skin infections caused by dermatophytes or Candida species, with inflammatory symptoms suitable for treatment with an ointment. Pregnant women and patients with bacterial or viral infections were excluded. The study did not specify randomization, control, or blinding methods. A total of 107 patients were included in the analysis, with 34 treated for 2 weeks and 71 for 4 weeks. The diagnosis was confirmed by microscopic assessment of a KOH preparation. The treatment regimen consisted of 2% miconazole nitrate and 1% hydrocortisone in a fatty ointment, applied twice daily for up to 4 weeks. Efficacy endpoints included the global assessment of treatment results and symptom score reduction, while safety endpoints focused on the reporting of side

Created a chunk of size 1070, which is longer than the specified 1000


summary time: 28.058229207992554 seconds
summary_output Den Boer and Schuller (1980) conducted an open study to evaluate the effectiveness of Daktacort in treating inflamed skin lesions of mycotic and/or bacterial origin. The study included 20 patients with symptoms of a mycotic infection and marked inflammatory signs such as redness and itching. The median duration of symptoms was 2 years, with a range from 2 weeks to 20 years. Patients were treated with a cream containing 2% miconazole nitrate and 1% hydrocortisone, applied twice daily without the use of other medications for their condition. The treatment duration was 6 weeks. Efficacy was assessed by the presence of fungi and/or bacteria at the start, 5 days after, and 1 month after the end of therapy, as well as the evaluation of typical signs and symptoms at 2, 4, and 6 weeks of treatment. The global evaluation of treatment outcome was also performed at these intervals.

A clear improvement of all symptoms was observed within a f

In [7]:
summary_t3_dev_final_prompt = """Consider yourself as a medical writer.
    Your job is to summarize the given input text like a medical writer summary.
    The output summary must have 4 sections: Background,efficacy,safety and author conclusion sections.
    Must use the following information for author_name(publication_year): {cite_info}


    Background section:
    Must include the author name and the publication year in the following format: 'author_name(publication_year)'
    Must begin with "author_name(publication_year)" .Strictly not to make any changes in the author_name(publication_year).
    Include essential details like study aim,study design,randomization method,control,blinding,population,diagnosis of interest,diagnostic criteria,age,sex,treatment and exposure,number of participants per treatment,disease condition,dose,dosage form,dose frequency,mode of administration,treatment duration,efficacy endpoints,safety endpoints.
    Only include the mentioned essential details if they are available.
    Include the following details only if present: The total number of subjects that were included in the analysis(with split-ups),subset of subjects that were specifically analyzed for safety-related outcomes or parameters(with split-ups), subset of subjects that were analyzed for efficacy-related outcomes(with split-ups) maintaining their verbatim representation.
    Strictly Important: If any detail is not present/explicitly present, the prompt should ignore it, and there should be no generation of model content, such as 'The study does not specify the country where it was conducted'
    Do not explicitly mention the names of the essential details in the output summary like 'The diagnosis of interest was', 'The study design was'.
    Highly Important: If the aim or study design is not explicitly presented, generate and include them in the background section.
    Must retain the intervention details like treatment regimen with dose in the output summary.
    Strictly important to include the demographics of study participants(including mean/median age,percentage of participants split),treatment regimen,inclusion and exclusion criteria as originally presented in the input text.
    Avoid including information about how the study is conducted or the study procedure.
    Include information about medical procedure.
    Highly Important: Include the detailed information(only if available) about how the randomization process was conducted,the method used to assign patients to different treatment arms, maintaining their verbatim representation.
    Must include the data enclosed with brackets.

    Efficacy section:
    Extract the efficacy results including the results of the efficacy endpoints/effectiveness evaluation and present it in an elaborated manner.
    Highly Important: Include percentages, absolute values (if available), p-values(if available), confidence intervals(if available), odds ratio(if available), hazard ratio(if available), relative risk(if available). Must include the data enclosed within brackets.

    Safety section:
    Extract the safety results including the results of the safety endpoints/safety evaluation and present it in an elaborated manner.
    Highly Important: Include percentages, absolute values (if available), p-values(if available), confidence intervals(if available), odds ratio(if available), hazard ratio(if available), relative risk(if available). Must include the data enclosed within brackets.

    Author conclusion:
    Read the entire input text and find out the author conclusion.
    When presenting the author conclusion, include the phrase 'The authors concluded that' at the beginning.
    Highly Important: If the author conclusion is not present explicitly, generate abstractively based on understanding.

    ***Strictly adhere to the following guidelines:***
    Highly important to make sure not to include the section names in the output summary.
    Highly important to make sure not to include headers, groups and number listing in the output summary.
    Highly important to make sure not to give introduction sentence,conclusion sentence while generating summary.
    Safety results should follow the presentation of efficacy results, and the author conclusion should be presented at the end.
    Highly Important: Reproduce sentences in the paragraph mostly verbatim to maintain language flow.
    Ensure there is no repetition of information between different parts of the output summary.
    Strictly avoid including reference information like references to tables, paragraphs, figures, etc.
    Do not add newline characters in any other part of the output summary.
    Incorporate very concise form of study limitations from the input text into the output summary, only if available. Don't generate on your own.
    Organize the information in chronological order.
    Strictly avoid providing any false information.
    Highly important to differentiate the efficacy,safety and author conclusion sections by adding a newline character('\n\n').
"""


with mlflow.start_run(run_name='Manual_logging',
                      tags={"mlflow.source.git.commit": 1},
                      log_system_metrics=True) as run:
    mlflow.log_text(summary_t3_dev_final_prompt, "prompt.txt")
    mlflow.log_table(data=xl_pd.to_dict(orient='list'), artifact_file="prompt_token.json")

2024/03/25 11:17:40 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2024/03/25 11:17:40 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/03/25 11:17:40 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [17]:
from mlflow.metrics.metric_definitions import _token_count_eval_fn

token =  _token_count_eval_fn([summary_t3_dev_final_prompt]).scores[0]
print(token)

922


In [22]:
import tiktoken

    # ref: https://github.com/openai/tiktoken/issues/75
encoding = tiktoken.get_encoding("cl100k_base")

token = len(encoding.encode(input_text))
print(token)


1933


In [28]:
xl_pd['prompt_token'] = _token_count_eval_fn([summary_t3_dev_final_prompt]).scores[0]
xl_pd['content_token'] = xl_pd['content'].apply(lambda x: _token_count_eval_fn([x]).scores[0])
xl_pd['summary_token'] = xl_pd['summary'].apply(lambda x: _token_count_eval_fn([x]).scores[0])
xl_pd['Total_token'] = xl_pd['prompt_token'] + xl_pd['content_token'] + xl_pd['summary_token']

In [29]:
xl_pd.to_dict(orient='list')

{'article': ['Aussems 1974_LMD004944.pdf',
  'Bjornberg 1986_LMD052123.pdf',
  'Clinical Progress Report R14889_11_1971_LMD005591.pdf',
  'Clinical Progress Report_R14889plus35_1973_LMD007028.pdf',
  'Clinical Research Report R148891NL_1985_LMD046244.pdf',
  'Den Boer 1980_LMD018448.pdf',
  'Internal Report Project 1355.00_Taplin 1976_LMD012214.pdf',
  'Mertens 1976_LMD010427.pdf'],
 'summary': ['Aussems and Van Cutsem (1974) conducted an open trial with 157 outpatients presenting with 176 lesions of apparently fungal origin associated with inflammatory symptoms. The study aimed to evaluate the efficacy and safety of Daktacort cream, which contains miconazole nitrate 2% and hydrocortisone 1%. Patients applied the cream twice daily for a median duration of four weeks, with extremes ranging from one week to 12 weeks. The clinical results were assessed at two-week intervals and scored as cure (++), improvement (+), or insufficient clinical results (0). The population included patients wit

In [32]:
xl_pd

,article,summary,Time taken,summary_token,content,content_token,prompt_token,Total_token
0,Aussems 1974_LMD004944.pdf,Aussems and Van Cutsem (1974) conducted an ope...,41.209141,421,Patients and Methods \nOne hundred and fifty-s...,1230,922,2573
1,Bjornberg 1986_LMD052123.pdf,Bjornberg and Tegner (1986) conducted a double...,26.505733,359,TREATMENT OF TINEA WITH MICONAZOLE AND MICONAZ...,1413,922,2694
2,Clinical Progress Report R14889_11_1971_LMD005...,Botter (1971) conducted a study on 30 out-pati...,24.766840,299,Materials and Method \nThe study was performed...,747,922,1968
3,Clinical Progress Report_R14889plus35_1973_LMD...,Rieth (1973) conducted a study to evaluate the...,34.152289,441,JANSSEN RESEARCH PRODUCTS INFORMATION SERVICE ...,969,922,2332
4,Clinical Research Report R148891NL_1985_LMD046...,Staal (1985) conducted a study to evaluate the...,45.991093,397,SUMMARY. \nThe efficacy and tolerance of Dakta...,2605,922,3924
5,Den Boer 1980_LMD018448.pdf,Den Boer and Schuller (1980) conducted an open...,35.539358,450,TRANSLATION FROM THE DUTCH \nTREATMENT OF INFL...,1263,922,2635
6,Internal Report Project 1355.00_Taplin 1976_LM...,Taplin (1976) conducted a study to evaluate th...,32.806528,317,"JohnsonJJhmen \nNew Brunswick. NJ. October 5, ...",1200,922,2439
7,Mertens 1976_LMD010427.pdf,Mertens et al (1976) conducted a 4-week double...,28.477100,527,"A Double-Blind Study Comparing Daktacort®, , M...",1933,922,3382
